In [1]:
import pandas as pd
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import seawater
import netCDF4 as nc
import sys
sys.path.append('/data/tjarniko/mocsy')
import mocsy
import gsw
import warnings
from salishsea_tools import (
    nc_tools,
    viz_tools,
    geo_tools,
    tidetools
)

## calculate observational omega for all datasets and save pandas dataframe (append to previous one) - also create dataset f

In [13]:
fname = './Datasets/2015dat_fromdf.csv'
dset = 'D15'

fnames = ['./Datasets/2015dat_fromdf.csv','./Datasets/2014dat_fromdf.csv',\
          './Datasets/GRL_fromdf.csv', './Datasets/combined_fromdf.csv']
dsets = ['D15','D14','GRL','COMB']


for i in range(0,4):
    fname = fnames[i]
    dset = dsets[i]
    df = pd.read_csv(fname)
    crid = df['crid'].to_numpy()
    dic = df['dic'].to_numpy()
    alk = df['alk'].to_numpy()
    dic_qf = df['dic_qf'].to_numpy()
    alk_qf = df['alk_qf'].to_numpy()
    lat = df['lat'].to_numpy()
    lon = df['lon'].to_numpy()
    S = df['S'].to_numpy()
    T = df['T'].to_numpy()
    P = df['P'].to_numpy()
    mon = df['mon'].to_numpy()
    day = df['day'].to_numpy()


    tzero = np.zeros_like(alk)
    dens = seawater.dens(S,T,P)
    # CONVERT FROM UMOL TO MMOL/M3
    dic_conv=dic*dens/1000 * 1e-3
    ta_conv = alk*dens/1000 * 1e-3
    tpressure = tzero +1 

    response_tup = mocsy.mvars(temp=T, sal=S, alk=ta_conv, dic=dic_conv,
                       sil=tzero, phos=tzero, patm=tpressure, depth=P, lat=tzero,
                        optcon='mol/m3', optt='Tinsitu', optp='m',
                        optb = 'l10', optk1k2='m10', optkf = 'dg', optgas = 'Pinsitu')
    pH,pco2,fco2,co2,hco3,co3,OmegaA_OBS,OmegaC,BetaD,DENis,p,Tis = response_tup

    df["OmegaA"] = OmegaA_OBS

    #while we're at it, let's add model i and model j to all of these datasets!
    mesh = nc.Dataset('/data/tjarniko/MEOPAR/grid/bathymetry_201702.nc')
    nav_lon = mesh.variables['nav_lon'][:]
    nav_lat = mesh.variables['nav_lat'][:]

    obs_j = np.zeros_like(lat); obs_i = np.zeros_like(lat)
    for w in range(0,len(lat)):
        j, i = geo_tools.find_closest_model_point(lon[w],lat[w],nav_lon,nav_lat)
        obs_j[w] = j
        obs_i[w] = i

    df["obs_j"] = obs_j; 
    df["obs_i"] = obs_i; 
    df.to_csv(fname, index=False)


    ### create a subset that's properly masked

    mask = (((dic_qf==2) | (dic_qf==6)) & \
        ((alk_qf==2) | (alk_qf==6)) & \
        (dic > 0) & (alk >0))
    crid_m = crid[mask]
    dic_m = dic[mask]
    alk_m = alk[mask]
    dic_qf_m = dic_qf[mask]
    alk_qf_m = alk_qf[mask]
    lat_m = lat[mask]
    lon_m = lon[mask]
    S_m = S[mask]
    T_m = T[mask]
    P_m = P[mask]
    mon_m = mon[mask]
    day_m = day[mask]
    obs_j_m = obs_j[mask]
    obs_i_m = obs_i[mask]
    OmegaA_OBS_m = OmegaA_OBS[mask]

    dens = seawater.dens(S_m,T_m,P_m)
    ta_conv=alk_m*dens/1000
    dic_conv=dic_m*dens/1000

    loc_tag = np.zeros_like(dic_m)
    for w in range(0, len(dic_m)):    
        x = obs_i_m[w]
        y = obs_j_m[w]

        #jdf is 1
        if  (x <200) & (y>220) & (y<500) :
            loc_tag[w] = 1
            #n strait
        if  (y >= 550):
            loc_tag[w] = 2
            #central strait
        if (x> 150) & (y>380) & (y<550):
            loc_tag[w] = 3
            #4 is haro/puget
        if ((x>150) & (y>320) & (y<370)):
            loc_tag[w] =4

    df2 = pd.DataFrame([crid_m, mon_m, day_m, lat_m, lon_m, P_m, T_m, S_m, dic_conv, ta_conv, dic_qf_m, alk_qf_m, obs_i_m, obs_j_m, OmegaA_OBS_m, loc_tag]).T
    df2.columns = ['crid','mon', 'day', 'lat', 'lon', 'P', 'T', 'S', 'dic', 'alk', 'dic_qf', 'alk_qf','obs_i', 'obs_j', 'OmegaA', 'loc_tag']
    dfnam = f'./extracted_calculated/OBS_{dset}_MASKED_umolL.csv'
    df2.to_csv(dfnam)

/home/tjarniko/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:62: RuntimeWarning: invalid value encountered in greater


## SUBSETS

In [8]:
df2.head()


,crid,mon,day,lat,lon,P,T,S,dic,alk,dic_qf,alk_qf,obs_i,obs_j,OmegaA,loc_tag
0,2012.19,4.0,2.0,49.03,-123.43,311.8,8.545,30.828,2169.189770,2210.921324,2.0,2.0,264.0,422.0,0.789514,3.0
1,2012.19,4.0,2.0,49.03,-123.43,251.3,8.323,30.726,2158.719688,2184.888534,2.0,2.0,264.0,422.0,0.700753,3.0
2,2012.19,4.0,2.0,49.03,-123.43,201.1,7.875,30.567,2138.406432,2177.459077,2.0,2.0,264.0,422.0,0.776961,3.0
3,2012.19,4.0,2.0,49.03,-123.43,150.4,7.890,30.435,2125.357392,2154.542462,6.0,6.0,264.0,422.0,0.720444,3.0
4,2012.19,4.0,2.0,49.03,-123.43,101.5,7.900,30.208,2105.791090,2128.185825,2.0,2.0,264.0,422.0,0.681664,3.0


In [12]:
df.tail()

,Unnamed: 0,crid,ln,stn,mon,day,lat,lon,P,T,S,dic,alk,dic_qf,alk_qf,OmegaA,obs_j,obs_i
1048,1048,2015.17,1.0,12.0,4.0,5.0,49.73,-124.68,175.6,9.317,30.481,NaN,NaN,9.0,9.0,1.000000e+20,647.0,168.0
1049,1049,2015.17,1.0,12.0,4.0,5.0,49.73,-124.68,200.7,9.438,30.608,2101.95,2087.77,2.0,2.0,5.073705e-01,647.0,168.0
1050,1050,2015.17,1.0,12.0,4.0,5.0,49.73,-124.68,251.0,9.384,30.736,2115.35,2126.71,2.0,2.0,6.302982e-01,647.0,168.0
1051,1051,2015.17,1.0,12.0,4.0,5.0,49.73,-124.68,300.8,9.361,30.800,NaN,NaN,9.0,9.0,1.000000e+20,647.0,168.0
1052,1052,2015.17,1.0,12.0,4.0,5.0,49.73,-124.68,351.6,9.307,30.835,2133.81,2122.55,2.0,2.0,5.138685e-01,647.0,168.0


### calculate corresponding model omega - depthspace

In [ ]:
# w = nc.Dataset('/data/tjarniko/MEOPAR/grid/mesh_mask201702.nc')
# depths = w['gdept_1d'][0][:]

# dsets = ['COMB','GRL','D14','D15']

# for dset in dsets:
#     print(dset)
#     for yr in range(2013,2018):
#         print(yr)
#         dsname = f'./ncs/YR{yr}_{dset}_depthspace_modeval.nc'    

#         w = nc.Dataset(dsname)
#         DIC = w['model_output']['DIC_int'][:]
#         TA = w['model_output']['TA_int'][:]    
#         sal = w['model_output']['sal_int'][:] 
#         temp = w['model_output']['temp_int'][:]
#         mod_dind = w['model_output']['model_d'][:]
#         mod_j = w['model_output']['model_j'][:]
#         mod_i = w['model_output']['model_i'][:]

#         mod_depths = np.zeros_like(mod_dind)

#         for i in range(0,len(mod_depths)):
#             mod_depths[i] = depths[int(mod_dind[i])]

#         P = mod_depths
#         tzero  = np.zeros_like(DIC)
#         tsra_psu = sal*35/35.16504
#         ttera_is = gsw.t_from_CT(sal,temp,P)
#         tpressure = tzero +1 

#         response_tup = mocsy.mvars(temp=ttera_is, sal=tsra_psu, alk=TA*1e-3, dic=DIC*1e-3,
#                            sil=tzero, phos=tzero, patm=tpressure, depth=P, lat=tzero,
#                             optcon='mol/m3', optt='Tinsitu', optp='m',
#                             optb = 'l10', optk1k2='m10', optkf = 'dg', optgas = 'Pinsitu')
#         pH,pco2,fco2,co2,hco3,co3,OmegaA_MOD,OmegaC,BetaD,DENis,p,Tis = response_tup

#         loc_tag = np.zeros_like(mod_depths)
#         for w in range(0, len(mod_depths)):    
#             x = mod_i[w]
#             y = mod_j[w]

#             #jdf is 1
#             if  (x <200) & (y>220) & (y<500) :
#                 loc_tag[w] = 1
#                 #n strait
#             if  (y >= 550):
#                 loc_tag[w] = 2
#                 #central strait
#             if (x> 150) & (y>380) & (y<550):
#                 loc_tag[w] = 3
#                 #4 is haro/puget
#             if ((x>150) & (y>320) & (y<370)):
#                 loc_tag[w] =4

#         df = pd.DataFrame([DIC,TA,sal,temp,mod_dind,mod_i,mod_j,OmegaA_MOD,loc_tag]).T
#         df.columns = ['dic', 'alk', 'sal', 'temp', 'mod_d', 'mod_i', 'mod_j','OmegaA','loc_tag']
#         dfname = f'./extracted_calculated/YR{yr}_{dset}_depthspace_modeloutput_with_Omega.csv'
#         df.to_csv(dfname)

In [ ]:
# w = nc.Dataset('/data/tjarniko/MEOPAR/grid/mesh_mask201702.nc')
# depths = w['gdept_1d'][0][:]

# dsets = ['COMB']

# for dset in dsets:
#     print(dset)
#     for yr in range(2015,2016):
#         print(yr)
#         dsname = f'./ncs/YR{yr}BR3_{dset}_depthspace_modeval.nc'    

#         w = nc.Dataset(dsname)
#         DIC = w['model_output']['DIC_int'][:]
#         TA = w['model_output']['TA_int'][:]    
#         sal = w['model_output']['sal_int'][:] 
#         temp = w['model_output']['temp_int'][:]
#         mod_dind = w['model_output']['model_d'][:]
#         mod_j = w['model_output']['model_j'][:]
#         mod_i = w['model_output']['model_i'][:]

#         mod_depths = np.zeros_like(mod_dind)

#         for i in range(0,len(mod_depths)):
#             mod_depths[i] = depths[int(mod_dind[i])]

#         P = mod_depths
#         tzero  = np.zeros_like(DIC)
#         tsra_psu = sal*35/35.16504
#         ttera_is = gsw.t_from_CT(sal,temp,P)
#         tpressure = tzero +1 

#         response_tup = mocsy.mvars(temp=ttera_is, sal=tsra_psu, alk=TA*1e-3, dic=DIC*1e-3,
#                            sil=tzero, phos=tzero, patm=tpressure, depth=P, lat=tzero,
#                             optcon='mol/m3', optt='Tinsitu', optp='m',
#                             optb = 'l10', optk1k2='m10', optkf = 'dg', optgas = 'Pinsitu')
#         pH,pco2,fco2,co2,hco3,co3,OmegaA_MOD,OmegaC,BetaD,DENis,p,Tis = response_tup

#         loc_tag = np.zeros_like(mod_depths)
#         print('walrus')
#         for w in range(0, len(mod_depths)):    
#             x = mod_i[w]
#             y = mod_j[w]

#             #jdf is 1
#             if  (x <200) & (y>220) & (y<500) :
#                 loc_tag[w] = 1
#                 #n strait
#             if  (y >= 550):
#                 loc_tag[w] = 2
#                 #central strait
#             if (x> 150) & (y>380) & (y<550):
#                 loc_tag[w] = 3
#                 #4 is haro/puget
#             if ((x>150) & (y>320) & (y<370)):
#                 loc_tag[w] =4

#         df = pd.DataFrame([DIC,TA,sal,temp,mod_dind,mod_i,mod_j,OmegaA_MOD,loc_tag]).T
#         df.columns = ['dic', 'alk', 'sal', 'temp', 'mod_d', 'mod_i', 'mod_j','OmegaA','loc_tag']
#         dfname = f'./extracted_calculated/YR{yr}BR3_{dset}_depthspace_modeloutput_with_Omega.csv'
#         df.to_csv(dfname)

In [ ]:
# w = nc.Dataset('/data/tjarniko/MEOPAR/grid/mesh_mask201702.nc')
# depths = w['gdept_1d'][0][:]

# dsets = ['COMB']

# for dset in dsets:
#     print(dset)
#     for yr in range(2015,2016):
#         print(yr)
#         dsname = f'./ncs/YR{yr}BR3_{dset}_depthspace_modeval.nc'    

#         w = nc.Dataset(dsname)
#         DIC = w['model_output']['DIC_int'][:]
#         TA = w['model_output']['TA_int'][:]    
#         sal = w['model_output']['sal_int'][:] 
#         temp = w['model_output']['temp_int'][:]
#         mod_dind = w['model_output']['model_d'][:]
#         mod_j = w['model_output']['model_j'][:]
#         mod_i = w['model_output']['model_i'][:]

#         mod_depths = np.zeros_like(mod_dind)

#         for i in range(0,len(mod_depths)):
#             mod_depths[i] = depths[int(mod_dind[i])]

#         P = mod_depths
#         tzero  = np.zeros_like(DIC)
#         tsra_psu = sal*35/35.16504
#         ttera_is = gsw.t_from_CT(sal,temp,P)
#         tpressure = tzero +1 

#         response_tup = mocsy.mvars(temp=ttera_is, sal=tsra_psu, alk=TA*1e-3, dic=DIC*1e-3,
#                            sil=tzero, phos=tzero, patm=tpressure, depth=P, lat=tzero,
#                             optcon='mol/m3', optt='Tinsitu', optp='m',
#                             optb = 'l10', optk1k2='m10', optkf = 'dg', optgas = 'Pinsitu')
#         pH,pco2,fco2,co2,hco3,co3,OmegaA_MOD,OmegaC,BetaD,DENis,p,Tis = response_tup

#         loc_tag = np.zeros_like(mod_depths)
#         print('walrus')
#         for w in range(0, len(mod_depths)):    
#             x = mod_i[w]
#             y = mod_j[w]

#             #jdf is 1
#             if  (x <200) & (y>220) & (y<500) :
#                 loc_tag[w] = 1
#                 #n strait
#             if  (y >= 550):
#                 loc_tag[w] = 2
#                 #central strait
#             if (x> 150) & (y>380) & (y<550):
#                 loc_tag[w] = 3
#                 #4 is haro/puget
#             if ((x>150) & (y>320) & (y<370)):
#                 loc_tag[w] =4

#         df = pd.DataFrame([DIC,TA,sal,temp,mod_dind,mod_i,mod_j,OmegaA_MOD,loc_tag]).T
#         df.columns = ['dic', 'alk', 'sal', 'temp', 'mod_d', 'mod_i', 'mod_j','OmegaA','loc_tag']
#         dfname = f'./extracted_calculated/YR{yr}BR3_{dset}_depthspace_modeloutput_with_Omega.csv'
#         df.to_csv(dfname)

## corresponding Omega in salinity

### both salinity, temperature, and pressure are taken from the observation 

In [ ]:
# w = nc.Dataset('/data/tjarniko/MEOPAR/grid/mesh_mask201702.nc')
# depths = w['gdept_1d'][0][:]

# dsets = ['COMB','GRL','D14','D15']

# for dset in dsets:
#     print(dset)
#     for yr in range(2013,2018):
#         print(yr)
#         dsname = f'./ncs/YR{yr}_{dset}_salspace_modeval.nc'    
#         fname = f'./extracted_calculated/OBS_{dset}_MASKED.csv'
#         df = pd.read_csv(fname)
#         S = df['S'].to_numpy()
#         P = df['P'].to_numpy()
#         T = df['T'].to_numpy()
#         w = nc.Dataset(dsname)
#         DIC = w['model_output']['DIC_int'][:]
#         TA = w['model_output']['TA_int'][:]  
#         mod_j = w['model_output']['model_j'][:]
#         mod_i = w['model_output']['model_i'][:]
#         lb_loc = w['model_output']['lb_loc'][:]
#         ub_loc = w['model_output']['ub_loc'][:]
        
#         tzero  = np.zeros_like(DIC)
#         tsra_psu = S
#         ttera_is = T
#         tpressure = tzero +1 

#         response_tup = mocsy.mvars(temp=ttera_is, sal=tsra_psu, alk=TA*1e-3, dic=DIC*1e-3,
#                            sil=tzero, phos=tzero, patm=tpressure, depth=P, lat=tzero,
#                             optcon='mol/m3', optt='Tinsitu', optp='m',
#                             optb = 'l10', optk1k2='m10', optkf = 'dg', optgas = 'Pinsitu')
#         pH,pco2,fco2,co2,hco3,co3,OmegaA_MOD,OmegaC,BetaD,DENis,p,Tis = response_tup

#         loc_tag = np.zeros_like(DIC)
#         for w in range(0, len(DIC)):    
#             x = mod_i[w]
#             y = mod_j[w]

#             #jdf is 1
#             if  (x <200) & (y>220) & (y<500) :
#                 loc_tag[w] = 1
#                 #n strait
#             if  (y >= 550):
#                 loc_tag[w] = 2
#                 #central strait
#             if (x> 150) & (y>380) & (y<550):
#                 loc_tag[w] = 3
#                 #4 is haro/puget
#             if ((x>150) & (y>320) & (y<370)):
#                 loc_tag[w] =4
        
#         df = pd.DataFrame([DIC,TA,mod_i,mod_j,lb_loc,ub_loc,OmegaA_MOD,loc_tag]).T
#         df.columns = ['dic', 'alk','mod_i', 'mod_j','lb_loc','ub_loc','OmegaA','loc_tag']
#         dfname = f'./extracted_calculated/YR{yr}_{dset}_salspace_modeloutput_with_Omega.csv'
#         df.to_csv(dfname)

In [ ]:

dsets = ['COMB']

for dset in dsets:
    print(dset)
    for yr in range(2015,2016):
        print(yr)
        dsname = f'./ncs/YR{yr}BR3_{dset}_salspace_modeval.nc'    
        fname = f'./extracted_calculated/OBS_COMB_MASKED_umolL.csv'
        df = pd.read_csv(fname)
        S = df['S'].to_numpy()
        P = df['P'].to_numpy()
        T = df['T'].to_numpy()
        w = nc.Dataset(dsname)
        DIC = w['model_output']['DIC_int'][:]
        TA = w['model_output']['TA_int'][:]  
        mod_j = w['model_output']['model_j'][:]
        mod_i = w['model_output']['model_i'][:]
        lb_loc = w['model_output']['lb_loc'][:]
        ub_loc = w['model_output']['ub_loc'][:]
        
        tzero  = np.zeros_like(DIC)
        tsra_psu = S
        ttera_is = T
        tpressure = tzero +1 

        response_tup = mocsy.mvars(temp=ttera_is, sal=tsra_psu, alk=TA*1e-3, dic=DIC*1e-3,
                           sil=tzero, phos=tzero, patm=tpressure, depth=P, lat=tzero,
                            optcon='mol/m3', optt='Tinsitu', optp='m',
                            optb = 'l10', optk1k2='m10', optkf = 'dg', optgas = 'Pinsitu')
        pH,pco2,fco2,co2,hco3,co3,OmegaA_MOD,OmegaC,BetaD,DENis,p,Tis = response_tup

        loc_tag = np.zeros_like(DIC)
        for w in range(0, len(DIC)):    
            x = mod_i[w]
            y = mod_j[w]

            #jdf is 1
            if  (x <200) & (y>220) & (y<500) :
                loc_tag[w] = 1
                #n strait
            if  (y >= 550):
                loc_tag[w] = 2
                #central strait
            if (x> 150) & (y>380) & (y<550):
                loc_tag[w] = 3
                #4 is haro/puget
            if ((x>150) & (y>320) & (y<370)):
                loc_tag[w] =4
        
        df = pd.DataFrame([DIC,TA,mod_i,mod_j,lb_loc,ub_loc,OmegaA_MOD,loc_tag]).T
        df.columns = ['dic', 'alk','mod_i', 'mod_j','lb_loc','ub_loc','OmegaA','loc_tag']

        
        
        dfname = f'./extracted_calculated/YR{yr}BR3_{dset}_salspace_modeloutput_with_Omega.csv'
        df.to_csv(dfname)

In [ ]:
P = 50
sal_rs = 30; temp = 12
ttera_is = gsw.t_from_CT(sal_rs,temp,P)
print(ttera_is)